In [1]:
import dcachefs
dcachefs.__version__

'0.1.3'

# Examples

The `dCacheFileSystem` class provides a python file-system interface for dCache. It builds on [Filesystem Spec](https://filesystem-spec.readthedocs.io) (in particular, on its `HTTPFileSystem` implementation) and it can be used as an independent object or via the general-purpose `fsspec` functions.

For the following examples, the file sytem is initially setup in the following way:

```
/
└── test
    ├── empty_testdir
    ├── testdir_1
    │   ├── file_1.txt
    │   └── file_2.txt
    └── testdir_2
        ├── file_1.txt
        └── file_2.txt
```

## Independent usage

In [2]:
from dcachefs import dCacheFileSystem

api_url = 'https://dcacheview.grid.surfsara.nl:22880/api/v1'

# read authentication token
with open('macaroon.dat') as f:
    token = f.read().strip()

fs = dCacheFileSystem(api_url=api_url, token=token)

The following methods are implemented via the [dCache API](https://dcache.org/old/manuals/UserGuide-6.2/frontend.shtml):

In [3]:
fs.ls('/test/testdir_1', detail=False)

['/test/testdir_1/file_1.txt', '/test/testdir_1/file_2.txt']

In [4]:
fs.ls('/test/testdir_1/file_1.txt')

[{'name': '/test/testdir_1/file_1.txt',
  'size': 12,
  'type': 'file',
  'created': datetime.datetime(2022, 4, 28, 15, 42, 17, 566000),
  'modified': datetime.datetime(2022, 4, 28, 15, 42, 17, 586000)}]

In [5]:
fs.ls('/test/empty_testdir')

[]

In [6]:
fs.info('/test/testdir_1/')

{'name': '/test/testdir_1/',
 'size': 512,
 'type': 'directory',
 'created': datetime.datetime(2022, 4, 28, 15, 42, 17, 503000),
 'modified': datetime.datetime(2022, 4, 28, 15, 42, 17, 644000)}

In [7]:
fs.mv('/test/testdir_2/file_1.txt', '/test/testdir_2/file_renamed.txt')
fs.ls('/test/testdir_2/', detail=False)

['/test/testdir_2/file_2.txt', '/test/testdir_2/file_renamed.txt']

In [8]:
fs.rm('/test/testdir_2/file_2.txt')
fs.ls('/test/testdir_2/', detail=False)

['/test/testdir_2/file_renamed.txt']

In [9]:
fs.exists('/test/testdir_1/file_1.txt')

True

In [10]:
fs.exists('/test/testdir_1/nonexistent_file.txt')

False

In [11]:
fs.isfile('/test/testdir_1/file_1.txt')

True

In [12]:
fs.isdir('/test/testdir_1/')

True

In [13]:
fs.created('/test/testdir_1/file_1.txt')

datetime.datetime(2022, 4, 28, 15, 42, 17, 566000)

In [14]:
fs.modified('/test/testdir_1/file_1.txt')

datetime.datetime(2022, 4, 28, 15, 42, 17, 586000)

In [15]:
fs.size('/test/testdir_1/file_1.txt')

12

In [16]:
fs.glob('/test/testdir_*/file_*.txt')

['/test/testdir_1/file_1.txt',
 '/test/testdir_1/file_2.txt',
 '/test/testdir_2/file_renamed.txt']

In [17]:
for root, _, _ in fs.walk('/test'):
    print(root)

/test
/test/testdir_1
/test/empty_testdir
/test/testdir_2


In [18]:
fs.find('/test')

['/test/testdir_1/file_1.txt',
 '/test/testdir_1/file_2.txt',
 '/test/testdir_2/file_renamed.txt']

In [19]:
fs.du('/test') # bytes

36

In [20]:
fs.checksum('/test/testdir_1/file_1.txt')

29852467411734638623150000865252066217

The following methods, which involve reading/writing files from/to dCache, require the WebDAV door to be specified via a separate input argument when instantiating the file-system object:

In [21]:
webdav_url = 'https://webdav.grid.surfsara.nl:2880'
fs = dCacheFileSystem(api_url=api_url, webdav_url=webdav_url, token=token)
fs.cat('/test/testdir_1/file_1.txt')

b'Hello world!'

In [22]:
local_path = './file.txt'
fs.download('/test/testdir_1/file_1.txt', local_path)

# check local copy
!cat $local_path

Hello world!

In [23]:
remote_path = '/test/testdir_2/file_uploaded.txt'
fs.upload(local_path, remote_path)

# check remote copy
fs.cat(remote_path)

b'Hello world!'

In [24]:
with fs.open('/test/testdir_1/file_1.txt', 'rb') as f:
    content = f.read()
content

b'Hello world!'

In [25]:
path = '/test/testdir_2/file_written.txt'
with fs.open(path, 'wb') as f:
    f.write(b'Hello world!')
fs.cat(path)

b'Hello world!'

## Usage via `fsspec`

Once imported, `dcachefs` registers itself as the `fsspec` implementation for the "dcache" protocol. This means that all `fsspec` methods on URL-paths of the following form will be dealt via the `dCacheFileSystem`:
```
dcache://path/to/file/or/dir
```
Parameters like the API URL or WebDAV door can be passed as input arguments to the `fsspec` functions or specified in a .ini or .json configuration file in the directory file `${HOME}/.config/fsspec/` (see the [section of the fsspec documentation on configuration](https://filesystem-spec.readthedocs.io/en/latest/features.html#configuration)).

In [26]:
import fsspec

uri = 'dcache://test/testdir_1/file_1.txt'

with fsspec.open(uri, token=token, api_url=api_url, webdav_url=webdav_url) as f:
    content = f.read()
content

b'Hello world!'

One can also temporarily register the class as the default implementation for the HTTPS protocol, e.g. to deal with URL-paths that include the WebDAV door:

In [27]:
# URI should include protocol and WeDAV door
uri = 'https://webdav.grid.surfsara.nl:2880/test/testdir_1/file_1.txt'

with dcachefs.register_implementation(protocol='https'):
    # no need to specify here the webdav_url - it's already part of the URI
    with fsspec.open(uri, token=token, api_url=api_url) as f:
        content = f.read()
content

b'Hello world!'